In [1]:
library(bigrquery)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Store the project ID
projectid = "yhcr-prd-phm-bia-core"

In [3]:
# CVD Episodes
sql3 <- "SELECT *  FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_AH.Cardiovascular_Diseases_Episodes`"
tb3 <- bq_project_query(projectid, sql3)
cvd_ep <-bq_table_download(tb3)

# Diabetes
sql1 <- "SELECT *  FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_AH.Diabetes`"
tb1 <- bq_project_query(projectid, sql1)
diabetes <-bq_table_download(tb1)

In [5]:
colnames(cvd_ep)

[1] "person_id"                     "age_angina_diag"              
 [3] "date_angina_diagnosis"         "date_angina_departure"        
 [5] "has_angina"                    "los_angina_days"              
 [7] "age_cardiomyopathy_diag"       "date_cardiomyopathy_diagnosis"
 [9] "date_cardiomyopathy_departure" "has_cardiomyopathy"           
[11] "los_cardiomyopathy_days"       "age_heart_failure_diag"       
[13] "date_heart_failure_diagnosis"  "date_heart_failure_departure" 
[15] "has_heart_failure"             "los_heart_failure_days"       
[17] "age_hypertension_diag"         "date_hypertension_diagnosis"  
[19] "date_hypertension_departure"   "has_hypertension"             
[21] "los_hypertension_days"         "age_ihd_diag"                 
[23] "date_ihd_diagnosis"            "date_ihd_departure"           
[25] "has_ihd"                       "los_ihd_days"                 
[27] "age_mi_diag"                   "date_mi_diagnosis"            
[29] "date_mi_departure"             "has_mi"                       
[31] "los_mi_days"                   "age_pvd_diag"                 
[33] "date_pvd_diagnosis"            "date_pvd_departure"           
[35] "has_pvd"                       "los_pvd_days"                 
[37] "age_stroke_diag"               "date_stroke_diagnosis"        
[39] "date_stroke_departure"         "has_stroke"                   
[41] "los_stroke_days"               "age_valve_disease_diag"       
[43] "date_valve_disease_diagnosis"  "date_valve_disease_departure" 
[45] "has_valve_disease"             "los_valve_disease_days"       
[47] "has_cvd"                       "cvd_number"                   
[49] "in_data"

In [12]:
# Keep only relevant variables, choose earliest CVD date and LOS for that CVD
cvd <- cvd_ep %>%
    mutate(earliest_cvd_date = pmin(date_angina_diagnosis, date_cardiomyopathy_diagnosis, date_heart_failure_diagnosis, date_hypertension_diagnosis, date_ihd_diagnosis, date_mi_diagnosis, date_pvd_diagnosis, date_stroke_diagnosis, date_valve_disease_diagnosis, na.rm = TRUE)) %>%
    mutate(age_earliest_cvd = case_when(
        earliest_cvd_date == date_angina_diagnosis ~ age_angina_diag,
        earliest_cvd_date == date_cardiomyopathy_diagnosis ~ age_cardiomyopathy_diag,
        earliest_cvd_date == date_heart_failure_diagnosis ~ age_heart_failure_diag,
        earliest_cvd_date == date_hypertension_diagnosis ~ age_hypertension_diag,
        earliest_cvd_date == date_ihd_diagnosis ~ age_ihd_diag,
        earliest_cvd_date == date_mi_diagnosis ~ age_mi_diag,
        earliest_cvd_date == date_pvd_diagnosis ~ age_pvd_diag,
        earliest_cvd_date == date_stroke_diagnosis ~ age_stroke_diag,
        earliest_cvd_date == date_valve_disease_diagnosis ~ age_valve_disease_diag)) %>%
    mutate(los_earliest_cvd = case_when(
        earliest_cvd_date == date_angina_diagnosis ~ los_angina_days,
        earliest_cvd_date == date_cardiomyopathy_diagnosis ~ los_cardiomyopathy_days,
        earliest_cvd_date == date_heart_failure_diagnosis ~ los_heart_failure_days,
        earliest_cvd_date == date_hypertension_diagnosis ~ los_hypertension_days,
        earliest_cvd_date == date_ihd_diagnosis ~ los_ihd_days,
        earliest_cvd_date == date_mi_diagnosis ~ los_mi_days,
        earliest_cvd_date == date_pvd_diagnosis ~ los_pvd_days,
        earliest_cvd_date == date_stroke_diagnosis ~ los_stroke_days,
        earliest_cvd_date == date_valve_disease_diagnosis ~ los_valve_disease_days)) %>%
    select(person_id, has_cvd, cvd_number, earliest_cvd_date, los_earliest_cvd, age_earliest_cvd)

In [15]:
# Keep relevant vars in diabetes 
diab <- diabetes %>%
    select(person_id, age_earliest_diab, earliest_diabetes_diag, any_diabetes)

In [17]:
head(cvd)
head(diab)

person_id,has_cvd,cvd_number,earliest_cvd_date,los_earliest_cvd,age_earliest_cvd
<int>,<int>,<int>,<date>,<int>,<int>
13377215,1,2,2013-04-21,12,81
12693795,1,2,2016-10-21,8,73
13674850,1,2,2020-10-27,37,50
12517678,1,2,2021-07-27,0,50
12457945,1,2,2014-11-21,15,86
13054895,1,2,2014-01-15,15,77


person_id,age_earliest_diab,earliest_diabetes_diag,any_diabetes
<int>,<int>,<date>,<int>
13548546,60,2007-01-11,1
12975170,80,2020-04-28,1
13450523,36,2011-10-13,1
12431500,75,2011-08-16,1
13235332,47,2007-07-05,1
12805324,39,2008-04-08,1


In [20]:
# Join cvd and diab - inner join, left join or full join?
db_cvd <- inner_join(diab, cvd, by = "person_id")
colnames(db_cvd)

[1] "person_id"              "age_earliest_diab"      "earliest_diabetes_diag"
[4] "any_diabetes"           "has_cvd"                "cvd_number"            
[7] "earliest_cvd_date"      "los_earliest_cvd"       "age_earliest_cvd"

In [25]:
sum(db_cvd$los_earliest_cvd == 1)

exclude_1_day <- db_cvd %>%
    filter(!los_earliest_cvd <= 1)

summary(exclude_1_day$los_earliest_cvd)
IQR(exclude_1_day$los_earliest_cvd)

[1] 1896

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  2.000   2.000   4.000   6.963   7.000 223.000 

[1] 5